In [ ]:
import tkinter as tk
from tkinter import scrolledtext
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")


In [ ]:
together_words = {"and", "both", "along with", "as well as", "in conjunction with", "together with"}
individual_words = {"or", "either", "one of", "neither", "nor", "any of", "each of"}

In [ ]:
class Preposition:
    def __init__(self, preposition):
        self.preposition = preposition
    
    def __repr__(self):
        return f"Preposition({self.preposition})"


class Noun:
    def __init__(self, names, preposition=None):
        self.names = names if isinstance(names, list) else [names] 
        self.preposition = preposition 
    
    def __repr__(self):
        if self.preposition:
            return f"Noun({', '.join(self.names)}, {self.preposition})"
        else:
            return f"Noun({', '.join(self.names)})"
    

class Verb:
    def __init__(self, actions, nouns):
        self.actions = actions if isinstance(actions, list) else [actions] 
        self.nouns = nouns  
    
    def __repr__(self):
        nouns_str = ', '.join([str(noun) for noun in self.nouns])
        return f"Verb({', '.join(self.actions)}, {nouns_str})"


preposition1 = Preposition("on")
noun1 = Noun(["table", "floor"], preposition1)
noun2 = Noun("chair")

# Create a list of verbs
verbs = Verb(["places", "moves"], [noun1, noun2])

# Print out the verb and its related nouns and prepositions
print(verbs)  # Output: Verb(places, moves, Noun(table, floor, Preposition(on)), Noun(chair))


Verb(places, moves, Noun(table, floor, Preposition(on)), Noun(chair))


In [ ]:
def analyse_tree(doc):

    all_data = []

    def collect_subj_conjuncts(subj, conj_nsubj):
        for child in subj.children:
            if child.dep_ == "conj":
                conj_nsubj.append(child.text)
                collect_subj_conjuncts(child, conj_nsubj)


    def collect_conjuncts(verb, root_subject, verb_data):
        for child in verb.children:
            if child.dep_ == "conj" and child.pos_ == "VERB":
                conj_nsubj = []

                # Check if the conjunct verb has its own subject
                for grandchild in child.children:
                    if grandchild.dep_ == "nsubj":
                        conj_nsubj.append(grandchild.text)
                        collect_subj_conjuncts(grandchild, conj_nsubj)

                # If no subject for the conjunct verb, add it to the root's verb_data
                if not conj_nsubj:
                    verb_data.append(child.text) 
                else:
                    all_data.append([[child.text], conj_nsubj])

                collect_conjuncts(child, root_subject, verb_data)

    for token in doc:
        if token.dep_ == "ROOT":
            verb_data = [token.text]  # Start with the root verb
            actors_list = []

            for child in token.children:
                if child.dep_ == "nsubj":
                    actors_list.append(child.text)
                    collect_subj_conjuncts(child, actors_list)
                    
            all_data.append([verb_data, actors_list])
            collect_conjuncts(token, actors_list, verb_data)

    return all_data


In [29]:
#tasks output
def print_tasks(doc):
    order_cnt = 0
    actors_actions = {}
    all_data = analyse_tree(doc)
    print(all_data)

    for verbs, actors in all_data:
        if len(actors) > 1:
            for verb in verbs:
                for actor in actors:
                    if actor not in actors_actions:
                        actors_actions[actor] = []
                    actors_actions[actor].append("BARRIER_" + ''.join(actors) + "_" + str(order_cnt))
                    actors_actions[actor].append(verb)
                    actors_actions[actor].append("SINC_"+ ''.join(actors))
                order_cnt+=1
        else:
            actor = actors[0]
            if actor not in actors_actions:
                    actors_actions[actor] = []
            for verb in verbs:
                    actors_actions[actor].append(verb)
    print(actors_actions)

input_text = "Robot A, Robot B and Robot C takes the pan and washes it, Robot B takes the pencil."
doc = nlp(input_text)
print_tasks(doc)



[[['takes', 'washes'], ['A', 'B', 'C']], [['takes'], ['B']]]
{'A': ['BARRIER_ABC_0', 'takes', 'SINC_ABC', 'BARRIER_ABC_1', 'washes', 'SINC_ABC'], 'B': ['BARRIER_ABC_0', 'takes', 'SINC_ABC', 'BARRIER_ABC_1', 'washes', 'SINC_ABC', 'takes'], 'C': ['BARRIER_ABC_0', 'takes', 'SINC_ABC', 'BARRIER_ABC_1', 'washes', 'SINC_ABC']}


In [28]:
# Test

#input_text = "Robot A and Robot B take the pot. Robot C takes the spoon. Robots A and Robot C help each other"
#input_text = "Robot A takes the pan, Robot B takes the pencil."
#input_text = "Robot A takes the pan and drops it. "
#input_text = "Robot A, Robot B and Robot C takes the pan, Robot B takes the pencil"
#input_text = "Robot A and Robot B take the pot. Robot C takes the spoon. Robot A and Robot C work together and Robot B is cleaning"

#WEIRD BEHAVIOUR in spacy
#input_text = "Robots A and Robot C help each other and Robot B is cleaning"
#input_text = "Robots A and Robot C help each other, Robot B cleans"

input_text = "Robot A, Robot B or Robot C take the bottle, while Robot C rests"

doc = nlp(input_text)
#options={"compact": True, "distance":60}
spacy.displacy.serve(doc, style="dep", auto_select_port=True)
print_tasks(doc)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5001 ...

Shutting down server on port 5001.
[[['take'], ['A', 'B', 'C']]]
{'A': ['BARRIER_ABC_0', 'take', 'SINC_ABC'], 'B': ['BARRIER_ABC_0', 'take', 'SINC_ABC'], 'C': ['BARRIER_ABC_0', 'take', 'SINC_ABC']}


In [ ]:
def display_input():
    input_text = entry.get()
    label.config(text=input_text)
    
    doc = nlp(input_text)
    result_text.delete(1.0, tk.END)
    
   
    subjects_info = analyse_tree(doc)
    # Insert the subjects and conjuncts information into the text area
    result_text.insert(tk.END, subjects_info)

In [ ]:
# Create the main window
root = tk.Tk()
label = tk.Label(root, text="Your input will appear here", font=('Arial', 14))
label.pack(pady=10)
entry = tk.Entry(root, font=('Arial', 14))
entry.pack(pady=10)
button = tk.Button(root, text="Analyze", command=display_input, font=('Arial', 14))
button.pack(pady=10)
result_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=15, font=('Arial', 12))
result_text.pack(pady=10)

root.mainloop()

# Part 1: Foundations of infovis 🚀

Let's get started with exploring some datasets with basic chart types!

👉 This part has two TODOs (1.1, 1.2).

To get started, here are the references for the data we use for some examples: 

- "Global air quality data provided by the World Health Organization" | [Source: WHO](https://www.who.int/data/gho/data/themes/air-pollution/who-air-quality-database)

- "WHO regional groupings" by income | [Source: WHO](https://cdn.who.int/media/docs/default-source/air-pollution-documents/air-quality-and-health/country-groupings-database-2022.pdf)

Let's load the dataset and have a first look: